### The Context Manager Decorator

You can use a decorator to encapsulate generators to make context managers.

This all seems very convoluted to me, but here are some examples

#### Code Examples

Typical setup of context managers using a generator is as follows:

In [ ]:
def gen(args):
    # set up happens here, or inside try
    try:
        yield obj # whatever normally gets returned by __enter__
    finally:
        # perform clean up code here

In [1]:
def open_file(fname, mode='r'):
    print('opening file...')
    f = open(fname, mode)
    try:
        yield f
    finally:
        print('closing file')
        f.close()

In [2]:
class GenContextManager:
    def __init__(self, gen):
        self.gen = gen
        
    def __enter__(self):
        print('calling next to get the yielded value from generator')
        return next(self.gen)
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        print('calling next to perform cleanup in generator')
        try:
            next(self.gen)
        except StopIteration:
            pass
        return False

In [3]:
file_gen = open_file('text.txt', 'w')
with GenContextManager(file_gen) as f:
    f.writelines('Sir Spamalot')

calling next to get the yielded value from generator
opening file...
calling next to perform cleanup in generator
closing file


In [4]:
def context_manager_dec(gen_fn):
    def helper(*args, **kwargs):
        gen = gen_fn(*args, **kwargs)
        ctx = GenContextManager(gen)
        return ctx
    return helper

In [5]:
@context_manager_dec
def open_file(fname, mode='r'):
    print('opening file...')
    f = open(fname, mode)
    try:
        yield f
    finally:
        print('closing file')
        f.close()
        
#This is the same as: open_file = context_manager_dec(open_file)

In [6]:
with open_file('text.txt') as f:
    print(f.readlines())

calling next to get the yielded value from generator
opening file...
['Sir Spamalot']
calling next to perform cleanup in generator
closing file


In [7]:
from contextlib import contextmanager

In [8]:
@contextmanager
def open_file(fname, mode='r'):
    print('opening file...')
    f = open(fname, mode)
    try:
        yield f
    finally:
        print('closing file')
        f.close()

In [9]:
with open_file('test.txt', 'r') as f:
    print(f.readlines())

opening file...
['line1\n', 'line2\n']
closing file


In [10]:
from time import perf_counter, sleep

In [11]:
@contextmanager
def timer():
    stats = dict()
    start = perf_counter()
    stats['start'] = start
    try:
        yield stats
    finally:
        end = perf_counter()
        stats['end'] = end
        stats['elapsed'] = end - start

In [12]:
with timer() as stats:
    sleep(2)

In [13]:
print(stats)

{'start': 4.8347805, 'end': 6.8352363, 'elapsed': 2.0004558}


In [14]:
import sys

In [15]:
@contextmanager
def out_to_file(fname):
    current_stdout = sys.stdout
    file = open(fname, 'w')
    sys.stdout = file
    try:
        yield None
    finally:
        file.close()
        sys.stdout = current_stdout

In [16]:
with out_to_file('test.txt'):
    print('line1')
    print('line2')

In [17]:
print('hello')

hello


In [18]:
with open('test.txt', 'r') as f:
    print(f.readlines())

['line1\n', 'line2\n']


In [19]:
from contextlib import redirect_stdout

In [20]:
with open('test.txt', 'w') as f:
    with redirect_stdout(f):
        print('Look on the bright side of life')

This above code redirects stdout to f